In [1]:
import requests
import pandas as pd

In [2]:
THE_FORGE = 10000002
JITA_TRADE_HUB = 60003760


def get_market_orders(
    region_id: int = THE_FORGE,
    location_id: int = JITA_TRADE_HUB,
    type_id: int | None = None,
    order_type: int | None = None,
) -> pd.DataFrame | None:
    url = f"https://esi.evetech.net/latest/markets/{region_id}/orders/"

    params = {"datasource": "tranquility"}
    params["order_type"] = order_type if order_type else "all"
    if type_id:
        params["type_id"] = type_id
    if location_id:
        params["location_id"] = location_id

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print("Failed to fetch data")
        return None

    return pd.DataFrame(response.json())

In [3]:
def get_ids(item_names: list[str]):
    url = "https://esi.evetech.net/latest/universe/ids/"

    headers = {"Content-Type": "application/json"}
    response = requests.post(url, headers=headers, json=item_names)

    if response.status_code != 200:
        print("Failed to fetch item IDs")
        return None

    data = response.json()
    return {item["name"]: item["id"] for item in data.get("inventory_types", [])}

In [4]:
def get_item_volume(type_id: int) -> pd.DataFrame | None:
    url = f"https://esi.evetech.net/latest/universe/types/{type_id}/"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to fetch volume for type_id {type_id}")
        return None

    data = response.json()
    return data.get("volume")

In [5]:
COLUMNS = [
    "type_name",
    "is_buy_order",
    "volume_remain",
    "price",
    "volume",
    "price_per_volume",
]


def get_jita_items_prices(item_ids: dict[str, int]) -> pd.DataFrame | None:
    out = None

    for id in item_ids.values():
        df = (
            get_market_orders(order_type="buy", type_id=id)
            .sort_values(by="price", ascending=False)
            .head(1)
        )
        df["volume"] = get_item_volume(id)
        df["price_per_volume"] = (df["price"] / df["volume"]).astype(int)

        if out is None:
            out = df
        else:
            out = pd.concat([out, df])

    swapped_ids = {val: key for key, val in item_ids.items()}
    out["type_name"] = out["type_id"].map(swapped_ids)

    return out[COLUMNS]

In [6]:
veldspar = [
    "Compressed Veldspar",
    "Compressed Concentrated Veldspar",
    "Compressed Dense Veldspar",
    "Compressed Stable Veldspar",
]

plagioclase = [
    "Compressed Plagioclase",
    "Compressed Azure Plagioclase",
    "Compressed Rich Plagioclase",
    "Compressed Sparkling Plagioclase",
]

scordite = [
    "Compressed Scordite",
    "Compressed Condensed Scordite",
    "Compressed Glossy Scordite",
    "Compressed Massive Scordite",
]

In [7]:
ids = get_ids(veldspar + plagioclase + scordite)
get_jita_items_prices(ids).sort_values(by="price_per_volume", ascending=False)

,type_name,is_buy_order,volume_remain,price,volume,price_per_volume
3,Compressed Glossy Scordite,True,1741853,45.83,0.0015,30553
22,Compressed Rich Plagioclase,True,183636,75.46,0.0035,21559
13,Compressed Azure Plagioclase,True,836148,73.56,0.0035,21017
35,Compressed Plagioclase,True,3399472,62.74,0.0035,17925
36,Compressed Massive Scordite,True,7223756,23.38,0.0015,15586
35,Compressed Condensed Scordite,True,2458428,22.94,0.0015,15293
35,Compressed Scordite,True,27905684,22.16,0.0015,14773
1,Compressed Stable Veldspar,True,999929,14.00,0.0010,14000
10,Compressed Sparkling Plagioclase,True,583398,47.16,0.0035,13474
24,Compressed Dense Veldspar,True,15499687,13.06,0.0010,13060
